In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [2]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (0, 0, 0)
r2 = (0, 1, 0)
r3 = (2, 3, 0)
r4 = (4, 1, 0)

floaters = np.hstack((r1, r2, r3, r4))

m = 4             # total atoms in the chain

In [3]:
# define parameters
beta = 1

In [4]:
# define time interval
t = np.linspace(0,6,20)

In [5]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((m,3))
    total_bending_force = np.zeros(m)
    
    # loop for computing bending force
    for i in range(1,m-1):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            
        except Exception as error:
            print("Caught exception: ", error)
            continue
            
        current_bending_force[i,0] += Eb_common*(A*Bxn - B*Axn)
        current_bending_force[i,1] += Eb_common*(A*Byn - B*Ayn)
        current_bending_force[i,2] += Eb_common*(A*Bzn - B*Azn)
            
        current_bending_force[i-1,0] += Eb_common*(A*Bxn0 - B*Axn0)
        current_bending_force[i-1,1] += Eb_common*(A*Byn0 - B*Ayn0)
        current_bending_force[i-1,2] += Eb_common*(A*Bzn0 - B*Azn0)
            
        current_bending_force[i+1,0] = Eb_common*(A*Bxn1 - B*Axn1)
        current_bending_force[i+1,1] = Eb_common*(A*Byn1 - B*Ayn1)
        current_bending_force[i+1,2] = Eb_common*(A*Bzn1 - B*Azn1)

    total_bending_force = np.hstack(current_bending_force)
    return total_bending_force

In [6]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

CPU times: user 31.4 ms, sys: 3.66 ms, total: 35.1 ms
Wall time: 41.7 ms


In [7]:
print(gFlow)

[[ 0.          0.          0.          0.          1.          0.          2.
   3.          0.          4.          1.          0.        ]
 [-0.23980325  0.034709    0.          0.08463433  1.14139466  0.
   1.95757517  2.57087136  0.          4.19759374  1.25302498  0.        ]
 [-0.39881853  0.09912456  0.          0.17136658  1.18102317  0.
   1.94744971  2.29426807  0.          4.28000224  1.4255842   0.        ]
 [-0.51022144  0.16937463  0.          0.24272586  1.17949629  0.
   1.94951546  2.09935671  0.          4.31798012  1.55177238  0.        ]
 [-0.58953905  0.23652624  0.          0.29734963  1.16089897  0.
   1.95696821  1.95500093  0.          4.33522121  1.64757386  0.        ]
 [-0.64654081  0.29696713  0.          0.33793636  1.13641078  0.
   1.96665704  1.84463504  0.          4.34194741  1.72198706  0.        ]
 [-0.6878573   0.34959336  0.          0.36774002  1.11129783  0.
   1.97692964  1.75846405  0.          4.34318764  1.78064476  0.        ]
 [-0.71807448

In [8]:
sx_n = gFlow[-1][3] - gFlow[-1][0]
sx_n1 = gFlow[-1][6] - gFlow[-1][3]

sy_n = gFlow[-1][4] - gFlow[-1][1]
sy_n1 = gFlow[-1][7] - gFlow[-1][4]

sz_n = gFlow[-1][5] - gFlow[-1][2]
sz_n1 = gFlow[-1][8] - gFlow[-1][5]

A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

print(A, B, A-B, A+B, 2*beta*(A-B)/(A+B))

2.17682782154 2.17649100792 0.000336813619154 4.35331882946 0.000154738778549


## Create Animation to Check that The Force is Correct

In [9]:
# define arrays of x, y and z coordinates to create an animated scatter plot
x = np.zeros((len(t),int(m)))
y = np.zeros((len(t),int(m)))
z = np.zeros((len(t),int(m)))
for i in range(len(t)):
    for j in range(int(m)):
        x[i,j] = gFlow[i,j*3]
        y[i,j] = gFlow[i,j*3+1]
        z[i,j] = gFlow[i,j*3+2]

In [10]:
%matplotlib inline

# create an ipyvolume.pylab figure for the simulation
p3.figure()

# plot a scatterplot of the arrays defined above in the figure
s1 = p3.scatter(x, y, z, marker='sphere', connected=True)
s2 = p3.plot(x, y, z)

# set the x, y and z limits of the figure
p3.xlim(-10,10)
p3.ylim(-10,10)
p3.zlim(0,2)

# add animation controls to the figure
p3.animation_control([s1, s2], interval=400)

# add sliders for parameters
beta_slider = widgets.IntSlider(min=0, max=15, step=1, value=beta, description='beta', continuous_update=False)

# create handler functions to change the values of the parameters globally when the sliders change
def on_beta_slider_change(change):
    global beta
    beta = change.new

# set the sliders to call their handler functions any time the sliders are moved
beta_slider.observe(on_beta_slider_change, names='value')

# display the VBox beneath the total system energy plot
label = widgets.Label('Parameter sliders:')
parameters = widgets.VBox([ipv.gcc(), label, beta_slider])
display(parameters)

VBox(children=(VBox(children=(Figure(animation=400.0, animation_exponent=1.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), connected=True, geo='sphere', size=array(2), size_selected=array(2.6), x=array([[ 0.        ,  0.        ,  2.        ,  4.        ],
       [-0.23980325,  0.08463433,  1.95757517,  4.19759374],
       [-0.39881853,  0.17136658,  1.94744971,  4.28000224],
       [-0.51022144,  0.24272586,  1.94951546,  4.31798012],
       [-0.58953905,  0.29734963,  1.95696821,  4.33522121],
       [-0.64654081,  0.33793636,  1.96665704,  4.34194741],
       [-0.6878573 ,  0.36774002,  1.97692964,  4.34318764],
       [-0.71807448,  0.38956718,  1.98689628,  4.34161103],
       [-0.74038293,  0.40559091,  1.99609863,  4.33869338],
       [-0.75701178,  0.41741722,  2.00433232,  4.33526224],
       [-0.76952641,  0.42620755,  2.01154236,  4.33177651],
       [-0.77903301,  0.43279396,  2.01775895,  4.3284801 ],
       [-0.78631905,  0.43777098,  2.02305756,  4.32549051],
       [-0.79194979,  0.44156403,  2.02753428,  4.32285148],
       [-0.79633459,  0.44447878,  2.03129098,  4.32056483],
       [-0.79977273,  0.44673623,  2.03442671,  4.31860979],
       [-0.80248517,  0.44849733,  2.03703309,  4.31695475],
       [-0.80463664,  0.44988028,  2.03919222,  4.31556414],
       [-0.80635115,  0.45097268,  2.04097604,  4.31440243],
       [-0.80772296,  0.45184007,  2.04244659,  4.3134363 ]]), y=array([[ 0.        ,  1.        ,  3.        ,  1.        ],
       [ 0.034709  ,  1.14139466,  2.57087136,  1.25302498],
       [ 0.09912456,  1.18102317,  2.29426807,  1.4255842 ],
       [ 0.16937463,  1.17949629,  2.09935671,  1.55177238],
       [ 0.23652624,  1.16089897,  1.95500093,  1.64757386],
       [ 0.29696713,  1.13641078,  1.84463504,  1.72198706],
       [ 0.34959336,  1.11129783,  1.75846405,  1.78064476],
       [ 0.394508  ,  1.08791887,  1.69023388,  1.82733925],
       [ 0.43235503,  1.06718538,  1.63569827,  1.86476132],
       [ 0.463977  ,  1.0492989 ,  1.59182991,  1.89489419],
       [ 0.490244  ,  1.03412785,  1.55638814,  1.91924001],
       [ 0.51197294,  1.02140017,  1.5276679 ,  1.938959  ],
       [ 0.52989422,  1.01080053,  1.50434528,  1.95495997],
       [ 0.54464253,  1.00201784,  1.48537757,  1.96796207],
       [ 0.55675958,  0.99476674,  1.46993506,  1.97853862],
       [ 0.56670233,  0.98879562,  1.45735283,  1.98714923],
       [ 0.57485303,  0.98388788,  1.44709525,  1.99416383],
       [ 0.58152968,  0.97985985,  1.43872932,  1.99988116],
       [ 0.58699563,  0.97655736,  1.43190402,  2.00454299],
       [ 0.59146835,  0.97385193,  1.42633431,  2.00834541]]), z=array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([[ 0.        ,  0.        ,  2.        ,  4.        ],
       [-0.23980325,  0.08463433,  1.95757517,  4.19759374],
       [-0.39881853,  0.17136658,  1.94744971,  4.28000224],
       [-0.51022144,  0.24272586,  1.94951546,  4.31798012],
       [-0.58953905,  0.29734963,  1.95696821,  4.33522121],
       [-0.64654081,  0.33793636,  1.96665704,  4.34194741],
       [-0.687857